In [152]:
import pandas as pd
import numpy as np
from numpy.random import choice
from math import sqrt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
# from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.metrics import accuracy_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import contingency_matrix
from scipy.optimize import linear_sum_assignment
from collections import Counter

In [158]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# # Ensure necessary NLTK resources are downloaded
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# Load the dataset
df = pd.read_csv('reviews_mixed.csv')

# Define lemmatization function
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    words = text.lower().split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(lemmatized_words)

# Apply lemmatization to the dataset
df['Text_Lemmatized'] = df['Text'].apply(lemmatize_text)

# Split the dataset into training and validation sets
training_input, validation_input, training_output, validation_output = train_test_split(
    df['Text_Lemmatized'], df['Sentiment'], test_size=0.2, random_state=42)

# Bag of Words
vectorizer = CountVectorizer()

# Transform the text data to feature vectors
training_embeddings = vectorizer.fit_transform(training_input).toarray()
validation_embeddings = vectorizer.transform(validation_input).toarray()

# Example text for prediction
text = ["By choosing a bike over a car, I'm reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I'm proud to be part of that movement."]
text_lemmatized = [lemmatize_text(t) for t in text]
text_embedding = vectorizer.transform(text_lemmatized).toarray()

# print("Training Embeddings Shape:", training_embeddings.shape)
# print("Validation Embeddings Shape:", validation_embeddings.shape)
# print("Text Embedding Shape:", text_embedding.shape)

In [154]:
df = pd.read_csv('reviews_mixed.csv')

def get_training_and_validation_datas(df: pd.DataFrame, training_size = 0.8):
    data_size = df.shape[0]
    indexes = [i for i in range(data_size)]
    training_index = np.random.choice(indexes,int(data_size*training_size))
    validation_index = [i for i in range(data_size) if not i in training_index]
    training_input = [df['Text'].iloc[index] for index in training_index]
    training_output = [df['Sentiment'].iloc[index] for index in training_index]
    validation_input = [df['Text'].iloc[index] for index in validation_index]
    validation_output = [df['Sentiment'].iloc[index] for index in validation_index]
    return training_input, training_output, validation_input, validation_output

training_input,training_output,validation_input, validation_output = get_training_and_validation_datas(df)

#bag of words:
vectorizer = CountVectorizer()
#tf-idf:
# vectorizer = TfidfVectorizer(max_features=50)


text = ["By choosing a bike over a car, I'm reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I'm proud to be part of that movement."]
# training_embeddings = vectorizer.fit_transform(training_input).toarray()
# validation_embeddings = vectorizer.transform(validation_input).toarray()

## KMEANS

In [159]:
kmeans = KMeans(n_clusters=2, n_init=10)
kmeans.fit(training_embeddings)

label_names = [name for name in set(training_output)]
validation_indexes = kmeans.predict(validation_embeddings)
computed_outputs = [label_names[value] for value in validation_indexes]

accuracy = accuracy_score(validation_output, computed_outputs)
print(f'Accuracy: {accuracy*100}%')

Accuracy: 47.61904761904761%


In [160]:
text = ["By choosing a bike over a car, I'm reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I'm proud to be part of that movement."]
input =  vectorizer.transform(text).toarray()
label = kmeans.predict(input)
print('Label:', label_names[label[0]])

Label: positive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertModel
import torch

# Load the dataset
df = pd.read_csv('reviews_mixed.csv')

def get_training_and_validation_datas(df: pd.DataFrame, training_size=0.8):
    data_size = df.shape[0]
    indexes = np.arange(data_size)
    training_index = np.random.choice(indexes, int(data_size * training_size), replace=False)
    validation_index = np.setdiff1d(indexes, training_index)
    
    training_input = df['Text'].iloc[training_index].tolist()
    training_output = df['Sentiment'].iloc[training_index].tolist()
    validation_input = df['Text'].iloc[validation_index].tolist()
    validation_output = df['Sentiment'].iloc[validation_index].tolist()
    
    return training_input, training_output, validation_input, validation_output

training_input, training_output, validation_input, validation_output = get_training_and_validation_datas(df)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_mean_embeddings(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encodings)
    # Take the mean of the token embeddings to create a single vector per sentence
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

# Get mean BERT embeddings for training and validation data
training_embeddings = get_mean_embeddings(training_input)
validation_embeddings = get_mean_embeddings(validation_input)

# Initialize and fit KMeans
kmeans = KMeans(n_clusters=2, n_init=10)
kmeans.fit(training_embeddings)

# Predict clusters for validation data
validation_indexes = kmeans.predict(validation_embeddings)
label_names = list(set(training_output))

# Map the cluster labels to the actual sentiment labels
computed_outputs = [label_names[idx] for idx in validation_indexes]

# Calculate accuracy
accuracy = accuracy_score(validation_output, computed_outputs)
print(f'Accuracy: {accuracy * 100}%')

# Predict the cluster label for new text input
new_text = ["By choosing a bike over a car, I'm reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I'm proud to be part of that movement."]
new_input_embeddings = get_mean_embeddings(new_text)
label = kmeans.predict(new_input_embeddings)
print('Label:', label_names[label[0]])


Accuracy: 52.38095238095239%
Label: negative


## AgglomerativeClustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

agglomerative = AgglomerativeClustering(n_clusters=2)
agglomerative.fit(training_embeddings)

label_names = [name for name in set(training_output)]
validation_indexes = agglomerative.fit_predict(validation_embeddings)
computed_outputs = [label_names[value] for value in validation_indexes]

accuracy = accuracy_score(validation_output, computed_outputs)
print(f'Accuracy: {accuracy * 100}%')

Accuracy: 35.78947368421053%


## GaussianMixture

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=2)
gmm.fit(training_embeddings)

label_names = [name for name in set(training_output)]
validation_indexes = gmm.predict(validation_embeddings)
computed_outputs = [label_names[value] for value in validation_indexes]

accuracy = accuracy_score(validation_output, computed_outputs)
print(f'Accuracy: {accuracy * 100}%')

Accuracy: 45.26315789473684%
